### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [74]:
PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}


class ReadableEntity:
    pass


class Journal(ReadableEntity):
    pass


class Book(ReadableEntity):
    pass


class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key]))
     
    
class LibraryJournal(Journal, Exporter):
    pass


class LibraryBook(Book, Exporter):
    pass

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [1]:
import sys
from collections import abc

In [2]:
basic_type = (int, float, str, bytes, range, bytearray)


In [3]:
def get_size(obj):
    size = sys.getsizeof(obj)

    if isinstance(obj, basic_type):
        print('size({}) += {}'.format(obj, size))
        return size
    
    elif isinstance(obj, abc.Sequence) or isinstance(obj, set):
        print('size({}) += {}'.format(obj, size))
        for i in obj:
            size += get_size(i)
            
    elif isinstance(obj, abc.Mapping):
        print('size({}) += {}'.format(obj, size))
        for k, v in obj.items():
            size += get_size(k) + get_size(v)
            
    if hasattr(obj, '__dict__'):
        size = get_size(obj.__dict__)
    return size

In [4]:
from collections import deque

In [5]:
class A():
    def __init__(self):
        self.a = 3
        

In [6]:
get_size('55')

size(55) += 51


51

In [8]:
get_size(bytearray(b'hello world!'))

size(bytearray(b'hello world!')) += 69


69

In [9]:
get_size(['55', [1,[1.8]]])

size(['55', [1, [1.8]]]) += 80
size(55) += 51
size([1, [1.8]]) += 80
size(1) += 28
size([1.8]) += 72
size(1.8) += 24


335

In [10]:
get_size(A())

size({'a': 3}) += 112
size(a) += 50
size(3) += 28


190

In [11]:
get_size(A().__dict__)

size({'a': 3}) += 112
size(a) += 50
size(3) += 28


190

### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [11]:
class FormatException(Exception):
    def __init__(self, message):
        pass

In [12]:
class Polynomial:
    
    def __init__(self, coefficients=None):
        """  Parameters
            --------
            coefficients: list of float or int. Coefficients before degrees of a polynomial from 0 to n 
        """
        if coefficients == None or len(coefficients) == 0:
            self.coefficients = [0]
        else:
            self.coefficients = list(coefficients)
        
        
        self.degree = len(self.coefficients) - 1

        if any([not isinstance(x, (float, int)) for x in self.coefficients]):
            raise FormatException('coefficients must be float or int')
    
    
    def __repr__(self):
        """
        method to return the canonical string representation 
        of a polynomial.
   
        """
        first_plus = 0
        res = ''
        
        if all([x == 0 for x in self.coefficients]):
            return '0'
        
        for power, coeff in enumerate(self.coefficients):
            
            coeff = int(coeff) if coeff%1==0 else round(coeff, 3)
            
            if power == 0 and coeff != 0:
                res += '{}'.format(coeff)
                first_plus += 1
            elif coeff > 0:
                res += '+'*(first_plus!=0)+'{}'.format(coeff)*(coeff != 1)+'x' + '**{}'.format(power)*(power >= 2)
                first_plus += 1
            elif coeff < 0:
                res += '-'+'{}'.format(-coeff)*(coeff != -1)+'x' + '**{}'.format(power)*(power >= 2)
                first_plus += 1
        return res
            
        
    def __call__(self, x):  
        """
        value of polynomial in x
        """
        res = 0
        for power, coeff in enumerate(self.coefficients):
            res += coeff * x**power
        return res 
    
    
    def mapping_coeffs(self, c1, c2, fill_value=0):
        """
        mapping by degrees
        """
        diff_len = len(c1) - len(c2)
        if diff_len > 0:
            c2 = c2 + [fill_value] * diff_len
        else:
            c1 = c1 + [fill_value] * (-diff_len)
        return c1, c2
    
    
    def __add__(self, polinom_other):
                
        if isinstance(polinom_other, (float, int)):
            polinom_other = Polynomial([polinom_other])
        elif not isinstance(polinom_other, Polynomial):
            raise FormatException("unsupported operand -: {} and {}".format(
                Polynomial, type(polinom_other)))
           
        c1 = self.coefficients
        c2 = polinom_other.coefficients   
        
        c1, c2 = self.mapping_coeffs(c1, c2)
        new_c = [sum(x) for x in zip(c1, c2)]
        return Polynomial(new_c)
    
    
    def __radd__(self, polinom_other):
        return self + polinom_other

    
    def __sub__(self, polinom_other):
        
        if isinstance(polinom_other, (float, int)):
            polinom_other = Polynomial([polinom_other])
        elif not isinstance(polinom_other, Polynomial):
            raise FormatException("unsupported operand -: {} and {}".format(
                Polynomial, type(polinom_other)))
            
        c1 = self.coefficients
        c2 = polinom_other.coefficients 
        
        c1, c2 = self.mapping_coeffs(c1, c2)
        new_c = [x[0]-x[1] for x in zip(c1, c2)]
        return Polynomial(new_c)
    
    
    def __rsub__(self, polinom_other):
        return self - polinom_other

    
    def __neg__(self):
        return Polynomial([-1*x for x in self.coefficients])
    
    
    def __pos__(self):
        return self
    
    
    def __mul__(self, polinom_other):
        if isinstance(polinom_other, (float, int)):
            polinom_other = Polynomial([polinom_other])
        elif not isinstance(polinom_other, Polynomial):
            raise FormatException("unsupported operand *: {} and {}".format(
                Polynomial, type(polinom_other)))

        c1 = self.coefficients
        c2 = polinom_other.coefficients 
        c1, c2 = self.mapping_coeffs(c1, c2)
        
        new_c = [0]*(len(c1)+len(c2)-1)
        
        for i1,  val1 in enumerate(c1):
            for i2, val2 in enumerate(c2):
                new_c[i1+i2] +=val1*val2

        return Polynomial(new_c)  
    
    
    def __rmul__(self, polinom_other):
        return self*polinom_other
    
    
    def effective_exponentiation(self, n):
        if n == 0: return 1
        else:
            if n % 2 == 1:
                return  self.effective_exponentiation( n - 1) * self
            else:
                b = self.effective_exponentiation( n // 2)
                return  b * b
    
    
    def __pow__(self, power):
        if not isinstance(power, (int)):
            raise FormatException("Power must be int")
        
        if power == 0:
            return  Polynomial([1])  
        
        self = self.effective_exponentiation(power)
        return self
              
        
    def _remove_first_zeros(self, li):
        i = True
        k = 0 
        while i & (len(li) > 0): 

            if li[0] == 0:
                li = li[1:]
                k += 1
            else:
                i=False

        return li, k


    def make_division(self, c1, c2):
    
        c1, _ = self._remove_first_zeros(c1)
        c2, _ = self._remove_first_zeros(c2)

        if len(c2) == 0:
            raise ZeroDivisionError()

        if len(c1) < len(c2):
            return Polynomial(), Polynomial(c1[::-1])

        new_c = []
        while len(c1) >= len(c2):
            new_c.append(c1[0] / c2[0])

            c1 = [c1[1:][i] - (new_c[-1] * c2[1:][i]) for i in range(len(c2) - 1)] + c1[len(c2):]

            if len(c1) < len(c2):
                return Polynomial(new_c[::-1]), Polynomial(c1[::-1])

            c1, k = self._remove_first_zeros(c1)

            if len(c1) == 0:
                new_c = new_c + [0]*(k-len(c2)+1)
                break
            else:
                new_c = new_c + [0]*k

        return Polynomial(new_c[::-1]), Polynomial(c1[::-1]).coefficients


    def __truediv__(self, polinom_other):
        if isinstance(polinom_other, (float, int)):
            polinom_other = Polynomial([polinom_other])
        elif not isinstance(polinom_other, Polynomial):
            raise FormatException("unsupported operand -: {} and {}".format(
                Polynomial, type(polinom_other)))
            
        return self.make_division(self.coefficients[::-1], polinom_other.coefficients[::-1])
    
    
    def derivative(self):
        derivative_coeffs = []
        
        for power, c in enumerate(self.coefficients):
            derivative_coeffs.append(power*c)
        return Polynomial(derivative_coeffs[1:])
    
    
    def integral(self):
        integral_coeffs = []
        
        for power, c in enumerate(self.coefficients):
            if c != 0:
                integral_coeffs.append(c / (power + 1) )
            else:
                integral_coeffs.append(0)
        return Polynomial([0] + integral_coeffs[:])


In [13]:
coef1 = [1,3, 5]
coef2 = [0, 1]

In [14]:
p1 = Polynomial(coef1)
p2 = Polynomial(coef2)
p3 = Polynomial([0])

In [15]:
p1.integral()

x+1.5x**2+1.667x**3

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [16]:
class MyRange:
    """
    class like range
    """
    def __init__(self, start, stop=None, step=1):
        
        if not isinstance(start, int):
            raise TypeError('{} object cannot be interpreted as an integerand', type(start))
        if step == 0:
            raise ValueError('my_range() arg 3 must not be zero')
            
        self.start = start
        
        if stop is None:
            self.stop = self.start 
            self.start = 0
        else:
            self.stop = stop
            
        self.step = step
                

    def __repr__(self):
        stop_str = (', ' +  str(self.step))*(self.step != 1)
        return 'my_range({}, {}{})'.format(self.start, self.stop, stop_str)
    
    def __len__(self):
        return (self.stop - self.start) // self.step + 1 *((self.stop - self.start) % self.step != 0)
    
    
    def __getitem__(self, ix):
        if ix > len(self) - 1:
            raise IndexError('my_range object index out of range')
    
        else:
            return list(self)[ix]
            
            
    def __iter__(self):
                
        if self.step > 0:
            while self.start < self.stop:
                yield self.start
                self.start += self.step
        elif self.step < 0:
            while self.start > self.stop:
                yield self.start
                self.start += self.step

In [17]:
list(MyRange(-1, -6, -2))

[-1, -3, -5]

In [18]:
list(range(-1, -6, -2))

[-1, -3, -5]

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [1]:
class PositiveDescriptors:
    def __init__(self, name=None):
        self.name = name
        
    def __set__(self, instance, value):
        if isinstance(value, int) and (0 <= value <= 32767):
            instance.__dict__[self.name] = value
        else:
            raise ValueError('Incorrect value')
            
    def __get__(self, instance, obj):
        return instance.__dict__[self.name]
            
    def __delete__(self, obj):
        del instance
     
    
class PositiveSmallIntegerField:
    descr = PositiveDescriptors('positive_number')
    
    def __init__(self, number):
        self.descr = number
        

In [2]:
a = PositiveSmallIntegerField(1)

In [3]:
a.descr

1

In [4]:
a.descr = -3

ValueError: Incorrect value

In [5]:
a.descr = 3

In [6]:
a.descr

3

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [7]:
import time

class Timer:
    def __init__(self):
        self.start_init = time.time()
        
    def __enter__(self):
        self.start_with = time.time()
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        print('Total time: {:.3f} sec.'.format(self.end - min(self.start_with, self.start_init)))

In [8]:
with Timer():
    time.sleep(3)

Total time: 3.000 sec.


In [9]:
t1 = Timer()
time.sleep(5)

with t1:
    time.sleep(3)

Total time: 8.001 sec.
